In [1]:
import pandas as pd
import numpy as np
import os
import time
import concurrent.futures
import multiprocessing
import xlrd
import torch
import random
import matplotlib.pyplot as plt
from scipy import stats
import scipy
import torch.nn as nn
import torch.nn.functional as tf
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import joblib
import matplotlib.pyplot as plt
import seaborn
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from typing import Optional


In [4]:
data_path = 'D:\\PycharmProjects\\pre_icu\\sepsis_0614\\Rcode\\SHENGLI\\MIMIC.xlsx'
data = pd.read_excel(data_path)
data.loc[((data['label']==1)&(data['death'].notnull())),'label']=2
data.loc[((data['Conscious']!='A')&(data['Conscious'].notnull())),'Conscious']='B'
data.head()
data=data.drop('death',1)
data=data.dropna()
data

C:\Users\USTCYuJi\AppData\Local\Temp\ipykernel_22220\2803483812.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data=data.drop('death',1)


,Sp,Hrate,Breath,BloodOxy,Temp,Whitecellcounts,label,Conscious,age
2,151.0,86.0,18.0,99.0,36.94,15.7,0.0,A,55.881486
3,73.0,55.0,19.0,100.0,36.50,17.0,0.0,A,46.275517
6,115.0,68.0,18.0,97.0,35.50,7.9,0.0,A,81.592179
8,129.0,72.0,14.0,93.0,36.61,4.3,0.0,A,77.917014
9,87.0,124.0,25.0,90.0,39.39,22.4,1.0,A,81.280232
...,...,...,...,...,...,...,...,...,...
53544,96.0,63.0,21.0,98.0,35.20,11.7,0.0,A,45.096905
53546,133.0,73.0,8.0,99.0,36.06,6.1,0.0,B,63.643884
53547,166.0,99.0,23.0,96.0,36.89,20.2,0.0,A,77.565907
53549,156.0,88.0,15.0,96.0,36.83,9.5,0.0,B,43.884161


In [6]:
#print(sum(data['label']==1),sum(data['label']==2))
data.to_csv('D:\\PycharmProjects\\pre_icu\\sepsis_0614\\Rcode\\SHENGLI\\MIMIC_data.csv',encoding='gbk')

In [2]:
#神经网络随机种子设定
def seed_torch(seed):

    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] =str(seed)#禁止hash随机化（unknown）

    torch.manual_seed(seed)#CPU种子
    torch.cuda.manual_seed(seed)#当前GPU种子
    torch.cuda.manual_seed_all(seed)#所有GPU种子

    torch.backends.cudnn.benchmark=False
    torch.backends.cudnn.deterministic=True#cuDNN是NVIDIA针对神经网络的加速库，牺牲少量精度换取运算效率

seed_number=2333

In [3]:
def NEWS_score(data):
    NEWS=pd.DataFrame(index=data.index,columns=['NEWS_score','呼吸_score','血氧_score','收缩压_score','心率_score','意识_score','体温_score','sepsis','危险程度'],data=0)
    #用for循环简单明了
    for i in data.index:
        if (data.loc[i,'呼吸']<=8)|(data.loc[i,'呼吸']>=25):
            NEWS.loc[i,'呼吸_score']=3
        if (data.loc[i,'呼吸']>=21)&(data.loc[i,'呼吸']<=24):
            NEWS.loc[i,'呼吸_score']=2
        if (data.loc[i,'呼吸']>=9)&(data.loc[i,'呼吸']<=11):
            NEWS.loc[i,'呼吸_score']=1

        if data.loc[i,'血氧']<=91:
            NEWS.loc[i,'血氧_score']=3
        if (data.loc[i,'血氧']>=92)&(data.loc[i,'血氧']<=93):
            NEWS.loc[i,'血氧_score']=2
        if (data.loc[i,'血氧']>=94)&(data.loc[i,'血氧']<=95):
            NEWS.loc[i,'血氧_score']=1

        if (data.loc[i,'收缩压']<=90)|(data.loc[i,'收缩压']>=220):
            NEWS.loc[i,'收缩压_score']=3
        if (data.loc[i,'收缩压']>=91)&(data.loc[i,'收缩压']<=100):
            NEWS.loc[i,'收缩压_score']=2
        if (data.loc[i,'收缩压']>=101)&(data.loc[i,'收缩压']<=110):
            NEWS.loc[i,'收缩压_score']=1

        if (data.loc[i,'心率']<=40)|(data.loc[i,'心率']>=131):
            NEWS.loc[i,'心率_score']=3
        if (data.loc[i,'心率']>=111)&(data.loc[i,'心率']<=130):
            NEWS.loc[i,'心率_score']=2
        if ((data.loc[i,'心率']>=41)&(data.loc[i,'心率']<=50))|((data.loc[i,'心率']>=91)&(data.loc[i,'心率']<=110)):
            NEWS.loc[i,'心率_score']=1

        if data.loc[i,'意识']!=1:
            NEWS.loc[i,'意识_score']=1

        if data.loc[i,'体温']<=35:
            NEWS.loc[i,'体温_score']=3
        if data.loc[i,'体温']>=39.1:
            NEWS.loc[i,'体温_score']=2
        if ((data.loc[i,'体温']>=35.1)&(data.loc[i,'体温']<=36))|((data.loc[i,'体温']>=38.1)&(data.loc[i,'体温']<=39)):
            NEWS.loc[i,'体温_score']=1

    NEWS['NEWS_score']=NEWS.apply(lambda x:x.sum(),axis=1)
    if 'sepsis' in data.columns:
        NEWS['sepsis']=data.loc[:,'sepsis']
    if '危险程度' in data.columns:
        NEWS['危险程度']=data.loc[:,'危险程度']
    return NEWS

# Ordinal Regression with Knowledge Distillation

## 0 序言

在一切开始之前，我们不得不介绍一下Ordinal Regression的背景和发展，这一读书结果我们将在读书笔记的PDF文件中展示。

## 1 数据载入与预处理

In [5]:
data_path = 'wkp_lqy_220508-6.CSV'
da = pd.read_csv(data_path,encoding='gbk')
choose_feature=['意识','体温','心率','呼吸','收缩压','血氧','白细胞计数']
choose_feature_c=['体温','心率','呼吸','收缩压','血氧','白细胞计数']
data=da.loc[:,choose_feature]
data['label']=da.loc[:,'sepsis']
data.loc[(da['sepsis']==1)&(da['危险程度']==1),'label']=2
data.head()
#print(sum(data['label']==1),sum(data['label']==2))
data.to_csv('C:\\Users\\USTCYuJi\\Desktop\\report-PPT\\221101\\data.csv',encoding='gbk')

In [10]:
#预处理：将意识A转为0，意识B&C转为1
data.loc[data['意识']=='A','意识']=0
data.loc[(data['意识']=='B')|(data['意识']=='C'),'意识']=1
data.head()



seed_torch(seed_number)
tfd_train = {
    0: 0.6,
    1: 0.6,
    2: 0.6
}

tfd_valid ={
    0: 0.5,
    1: 0.5,
    2: 0.5
}
def tsampling(group, tfd):
    name = group.name
    frac = tfd[name]
    return group.sample(frac=frac)

train = data.groupby('label',group_keys=False).apply(tsampling, tfd_train)
d_da0 = pd.concat([data, train,train]).drop_duplicates(keep= False)
valid = d_da0.groupby('label',group_keys=False).apply(tsampling, tfd_valid)
test = pd.concat([d_da0, valid,valid]).drop_duplicates(keep= False)
train

,意识,体温,心率,呼吸,收缩压,血氧,白细胞计数,label
3480,0,36.3,100,20,75,96,22.92,0
6440,0,36.4,93,20,162,97,11.14,0
4634,0,36.4,70,20,148,98,6.24,0
8005,0,36.3,87,28,126,98,2.94,0
9187,1,39.9,110,20,177,98,10.66,0
...,...,...,...,...,...,...,...,...
492,1,36.6,67,0,104,97,18.81,2
225,0,36.7,88,20,111,89,13.94,2
1101,0,37.2,112,21,114,88,1.55,2
527,0,36.0,144,22,74,91,30.40,2


## 2 基本原理与算法实现

### 2.1 Ordinal Regression

In [7]:
def _reduction(loss: torch.Tensor, reduction: str) -> torch.Tensor:
    """
    Reduce loss
    Parameters
    ----------
    loss : torch.Tensor, [batch_size, num_classes]
        Batch losses.
    reduction : str
        Method for reducing the loss. Options include 'elementwise_mean',
        'none', and 'sum'.
    Returns
    -------
    loss : torch.Tensor
        Reduced loss.
    """
    if reduction == 'elementwise_mean':
        return loss.mean()
    elif reduction == 'none':
        return loss
    elif reduction == 'sum':
        return loss.sum()
    else:
        raise ValueError(f'{reduction} is not a valid reduction')


def cumulative_link_loss(y_pred: torch.Tensor, y_true: torch.Tensor,
                         reduction: str = 'elementwise_mean',
                         class_weights: Optional[np.ndarray] = None
                         ) -> torch.Tensor:
    """
    Calculates the negative log likelihood using the logistic cumulative link
    function.
    See "On the consistency of ordinal regression methods", Pedregosa et. al.
    for more details. While this paper is not the first to introduce this, it
    is the only one that I could find that was easily readable outside of
    paywalls.
    Parameters
    ----------
    y_pred : torch.Tensor, [batch_size, num_classes]
        Predicted target class probabilities. float dtype.
    y_true : torch.Tensor, [batch_size, 1]
        True target classes. long dtype.
    reduction : str
        Method for reducing the loss. Options include 'elementwise_mean',
        'none', and 'sum'.
    class_weights : np.ndarray, [num_classes] optional (default=None)
        An array of weights for each class. If included, then for each sample,
        look up the true class and multiply that sample's loss by the weight in
        this array.
    Returns
    -------
    loss: torch.Tensor
    """
    eps = 1e-15
    # torch.clamp用于限制上下界，torch.gather中1表示对列做，取的index是y_true
    likelihoods = torch.clamp(torch.gather(y_pred, 1, y_true), eps, 1 - eps)
    neg_log_likelihood = -torch.log(likelihoods)

    if class_weights is not None:
        # Make sure it's on the same device as neg_log_likelihood
        class_weights = torch.as_tensor(class_weights,
                                        dtype=neg_log_likelihood.dtype,
                                        device=neg_log_likelihood.device)
        neg_log_likelihood *= class_weights[y_true]

    loss = _reduction(neg_log_likelihood, reduction)
    return loss


class CumulativeLinkLoss(nn.Module):
    """
    Module form of cumulative_link_loss() loss function
    Parameters
    ----------
    reduction : str
        Method for reducing the loss. Options include 'elementwise_mean',
        'none', and 'sum'.
    class_weights : np.ndarray, [num_classes] optional (default=None)
        An array of weights for each class. If included, then for each sample,
        look up the true class and multiply that sample's loss by the weight in
        this array.
    """

    def __init__(self, reduction: str = 'elementwise_mean',
                 class_weights: Optional[torch.Tensor] = None) -> None:
        super().__init__()
        self.class_weights = class_weights
        self.reduction = reduction

    def forward(self, y_pred: torch.Tensor,
                y_true: torch.Tensor) -> torch.Tensor:
        return cumulative_link_loss(y_pred, y_true,
                                    reduction=self.reduction,
                                    class_weights=self.class_weights)

In [17]:
fc_net=nn.Sequential(
        nn.Linear(len(choose_feature),3)
    ).double()
loss_func=CumulativeLinkLoss()
optimizer = torch.optim.Adam(fc_net.parameters(), lr=0.01)
epoch=100
x_train=torch.tensor(train.loc[:,choose_feature].values.astype("float"))
y_train=torch.tensor(train.loc[:,'label'].values.astype("int64"))
x_valid=torch.tensor(valid.loc[:,choose_feature].values.astype("float"))
y_valid=torch.tensor(valid.loc[:,'label'].values.astype("int64"))
x_test=torch.tensor(test.loc[:,choose_feature].values.astype("float"))
y_test=torch.tensor(test.loc[:,'label'].values.astype("int64"))
for i in range(3):
    logit_y = fc_net(x_train)
    pre_y=nn.functional.softmax(pre_y)
    print(pre_y)
    #logit_pre_y=lpre_y.exp()/(1+lpre_y.exp())#转到(0,1)之间
    #f_0=logit_pre_y[y_train==0]
    #f_0=torch.cat([1-f_0,f_0],dim=-1)
    #f_1=logit_pre_y[y_train==1]
    #f_1=torch.cat([1-f_1,f_1],dim=-1)
    #qg_0=torch.tensor([1-new_g0.values,new_g0.values]).t()
    #qg_1=torch.tensor([1-new_g1.values,new_g1.values]).t()
    #这两个loss的输入格式不一样，需要注意
    loss =loss_func(pre_y,y_train)#+LAMBDA0*torch.nn.functional.kl_div(input=f_0.log(),target=qg_0,reduction='batchmean')+LAMBDA1*torch.nn.functional.kl_div(input=f_1.log(),target=qg_1,reduction='batchmean')
    #epoch次数太多之后，f_0会非常接近0&1，这样在kl_div的计算中，f_0.log()会报错，此外，为了避免这种情况，还要对数据做标准化处理(总之数据标准化之后，就没出现数值错误)
    #optimizer.zero_grad()
    #loss.backward()
    #optimizer.step()
    print(loss)

tensor([[8.0463e-45, 1.0000e+00, 6.8837e-42],
        [8.0106e-52, 1.0000e+00, 6.2242e-39],
        [1.0513e-43, 1.0000e+00, 1.9327e-31],
        ...,
        [3.1740e-52, 1.0000e+00, 1.5631e-42],
        [2.0526e-57, 1.0000e+00, 1.0135e-55],
        [7.8681e-50, 1.0000e+00, 4.9958e-42]], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)


C:\Users\USTCYuJi\AppData\Local\Temp\ipykernel_36308\2138851499.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pre_y=nn.functional.softmax(pre_y)


RuntimeError: Index tensor must have the same number of dimensions as input tensor

## 3 结果展示与讨论
